# Imports

In [ ]:
import os
from PIL import Image
import pandas as pd
import shutil

# Input pipeline and preprocessing

## Convert images to jpg

In [ ]:
# for test dataset, create a "images" folder, which has the same structure as the original dataset, but jpg instead of ppm
# copy images and other files, do not delete the original dataset
path = "\\test\\images_ppm"

n = 0
images_path = os.path.join(os.path.dirname(path), "images")
os.makedirs(images_path, exist_ok=True)

# copy images
for file in os.listdir(path):
    src = os.path.join(path, file)
    # Nur Dateien verarbeiten, keine Verzeichnisse
    if not os.path.isfile(src):
        continue
    
    if file.endswith(".ppm"):
        dst = os.path.join(images_path, file.replace(".ppm", ".jpg"))
        img = Image.open(src)
        img.convert("RGB").save(dst, "JPEG")
    
    else:
        dst = os.path.join(images_path, file)
        shutil.copyfile(src, dst)
    
    n += 1
    if n % 100 == 0:
        print(f"Processed {n} files in {path}")

In [ ]:
# for the train_val dataset, imnages are in subfolders, where each subfolder is a class
# copy images and other files into one "images" folder, add class label to the filename (e.g. "class1_0001.jpg")
path = "\\train_val\\images_ppm"
n = 0

images_path = os.path.join(os.path.dirname(path), "images")
os.makedirs(images_path, exist_ok=True)

# copy images
for root, dirs, files in os.walk(path):
    for file in files:
        src = os.path.join(root, file)
        # Nur Dateien verarbeiten, keine Verzeichnisse
        if not os.path.isfile(src):
            continue
        
        if file.endswith(".ppm"):
            class_name = os.path.basename(root)
            dst = os.path.join(images_path, f"{class_name}_{file.replace('.ppm', '.jpg')}")
            img = Image.open(src)
            img.convert("RGB").save(dst, "JPEG")
        
        else:
            dst = os.path.join(images_path, file)
            shutil.copyfile(src, dst)
        
        n += 1
        if n % 100 == 0:
            print(f"Processed {n} files in {path}")

## Convert labels (?)

## YOLO dataset

In [ ]:
# convert csv annotations to YOLO format
# ...

In [ ]:
# create yaml file for yolo

# Training

In [ ]:
# ...

# Evaluation

In [ ]:
# ...